## 프로젝트 설계 리뷰(사례별)



1. 데이터 타입
    - erd 설계 시 데이터 타입 정하기 문제
    - 유저 id로 BIGINT를 사용한다면?
        - 2000경에 해당하는 수준의 유저를 받을 수 있게 됨..
        - 그냥 UINT를 사용한다면 42억으로 지구 전체 인구의 70퍼를 수용할 수 있음
        - 자료형의 크기는 B트리 구조 상 검색 속도에 차이를 야기함
        - B트리
            - 루트에 해당하는 페이지를 메모리에 올림
            - 이후 자식 노드의 값을 확인하고 차례대로 자식 노드로 페이지를 올림
            - 페이지의 크기는 16kb로 고정(설정으로 수정은 가능)
                - 속도가 가장 빠르기 때문
            - INT의 값은 4 바이트이고 데이터 주소가 12바이트
                - 한 행이 16 바이트의 크기를 가지기 때문에 한 페이지는 1024개 가량의 데이터를 가질 수 있음
            - BIGINT는 8바이트이기 때문에 절반 정도의 데이터밖에 가지지 못함
            - 한 페이지에 데이터를 더 많이 불러오기 때문에 검색(인덱싱 활용)에 있어서 작은 자료형이 유리함!!
    - 데이터 타입은 작을수록 빠르다
    - 사용 범위를 충분히 고려하여 가능한 작은 크기로 결정
    - 프라이머리 키의 타입은 특히 중요
        - 가급적 정수형
            - 이메일도 고유하니 이메일로 기본키를 설정할 수도 있음
            - But.. 속도가 늦어짐
        - Because AUTO_INCREMET 설정!
            - B트리 구조 상 정렬된 상태로 데이터를 저장
            - 정렬이 보장된 테이블에 대해서는 insert 시 데이터를 삽입할 위치를 찾는 시간이 거의 없음!
        - 쿼리 시 프라이머리 키가 사용됨
            - 다른 고유키로도 쿼리를 할 수 있음
    - REST Api에서 요청을 할 때
        - 가급적 프라이머리 키로 요청을 하는 게 베스트
2. 비정규화
    - 게시판과 댓글의 테이블을 생각할 때
        - 댓글은 외래키로 게시글 ID를 가짐
        - 게시판은 댓글의 갯수를 컬럼으로 가짐
    - 왜 댓글의 갯수가 컬럼으로?
        - 댓글 수는 댓글 테이블에 count를 하는 것으로 얻을 수 있음
    - 성능을 위하여 비정규화하고 컬럼을 추가한 것
    - 비정규화가 항상 좋은 것은 아님
        - ex) 게시판 테이블의 댓글 수 컬럼과 실제 댓글 테이블 담긴 댓글 갯수가 다를 때도 있음
        - 댓글이 추가되는 상황
            - 게시판에서는 update, 댓글에서는 insert가 일어남
            - 이 작업이 이뤄지는 순간에 원자성이 보장되지 않음
            - 데이터 오류 발생 순간이 존재한다는 것
    - 데이터와 JOIN이 많아지면 쿼리 성능을 흔히 걱정함
    - 그러나 `비정규화는 공짜가 아니다`
        - 비정규화 = 기술 부채
    - 성능은 높아지나 문제가 생길 여지
        
        → 유지 보수 비용 증가
        
    - 초반부터 비정규화하는 것은 자제
        - 운영하면서 성능이 느려질 때 고려
        - 차라리 메모리와 캐시를 더 확보하거나 쿼리만 최적화하는 것이 훨씬 더 이득일 수 있음
        - 비용을 고려하여 결정할 사항!!
    - 생각보다.. select count(*)는 느리지 않다
        - B트리 인덱싱 페이지의 값을 보고 카운팅!
    - 또 반복되는 쿼리는 디비에서 캐싱을 통해 성능 최적화 진행!
- 설계는 정답이 없다
- 악마는 디테일에 있다

2024.01.24 최인국 컨설턴트


#### 목표 : 설계 시 생각해봐야 할 다양한 측면 (정답은 없지만 고려해보기!!!)

1. 사례 1 - 데이터 타입
  
- UINT 최대 값 42억 ← 전 세계 인터넷 인구의 74%정도, but int보다 느림
- B Tree → 데이터 타입은 작을수록 빠르다 → 사용 범위를 충분히 고려 후, 가능한 작은 크기로 결정 → PK의 타입은 특히 중요함. (가급적 정수형, AUTO_INCREMENT 설정해야하기 때문) →퀴리 시 PK 우선 사용하기
- email, username(id)가 PK이면 찾는데 많은 시간이 걸린다. 그리고 보안상으로도 좋지 않음 ⇒ rest API 사용 시, pk로 검색하도록 하는것이 빠름!!

2. 사례 2 - 비정규화

- 게시글 - 댓글 (1:N)
  
- 댓글 수 ← 댓글 count하면 알 수 있음 (없어도 무방)
- 그럼에도 불구하고 게시글 table에 넣는다면 댓글수가 많을 때 시간이 걸리기 때문 ⇒ 비정규화의 목적 : 시간 단축

```sql
SELECT BOARD.*, COUNT(comment.id) AS comment_count FROM board

LEFT OUTER JOIN comment ON board.board_id = comment.board_id

GROUP BY board.board_id

ORDER BY board.id desc

LIMIT 10
```

- 비정규화 하게 되면 댓글 2개인데 1개로 표시되는 경우도 생김
- ^^ comment INSERT 된 후, board 테이블 update하지 않으면 값이 바뀌지 않기 때문!
- DATA가 많아지면, JOIN이 많아져 느려질 거라고 예상
- 비정규화는 공짜가 아님! 기술 부채임
- 성능 높이는 대신 문제가 발생할 여지를 남김 → 유지 보수 비용 증가 + QA팀에 시달림
- 미리 비정규화 자제 → 운영하다가 느려지면 판단 (HW 업그레이드 등의 선택지가 있음!)
- SELECT COUNT(*)는 느리지 않음 (캐시 있으면 더 빨라짐)
- ! SW 개발은 최소비용이 원칙임을 유의하자